# Query the NCBI STAT Metadata Tables to Search for Pathogens! 

## Overview

DNA sequence data are typically deposited into the [NCBI Sequence Read Archive](https://www.ncbi.nlm.nih.gov/sra). Each FASTQ file is assigned a taxon id (usually a species) defined by [NCBI taxonomy](https://www.ncbi.nlm.nih.gov/taxonomy). So, if you were to search SRA for a particular species, such as _[Mus musculus](https://www.ncbi.nlm.nih.gov/sra/?term=Mus+musculus)_ we can find the files associated with this taxon as defined by the sequence submitter. There are three possible issues with this approach. First, sometime people make mistakes about the taxon id of their sequence. They may have said the sequence was a mouse when it was actually a dog, and you won't know until you have analyzed that sequence. Second, most FASTQ files have mixed DNA sequence due to some level of contamination. If the mouse DNA came from a tail tip, the FASTQ will likely be full of microbial sequence as well as mouse DNA! Finally, many samples in SRA are metagenomic, and so you really have no idea what DNA is in there until you analyze it.

To address these issues, NCBI came up with a tool called the [SRA Taxonomy Analysis Tool]( https://www.ncbi.nlm.nih.gov/sra/docs/sra-taxonomy-analysis-tool/#:~:text=The%20NCBI%20SRA%20Taxonomy%20Analysis,from%20next%20generation%20sequencing%20runs), or STAT. STAT maps sequencing reads against a precomputed kmer dictionary and assigns reads to the lowest ambiguous taxonomic node (it is based on a known phylogeny). STAT is run for all SRA submissions, and the results are stored in Cloud-based metadata tables that can be queried using BigQuery. This table can then be matched to the SRA metadata tables to get robust information on each Accession. Here we walk through a basic STAT query for MPOX virus and teach you how to create your own queries. 

## Learning Objectives
+ Learn how to query BigQuery
+ Learn how to interact with and understand STAT tables

## Prerequisites
Make sure that the BigQuery API is enabled


## Get Started

### Import packages

In [ ]:
# Import the biquery api
from google.cloud import bigquery
import pandas as pd

In [ ]:
# Designate the client for the API
client = bigquery.Client(location="US")
print("Client creating using default project: {}".format(client.project))

### Define and run our query
Note that we are doing a few things here. First, we are going to query the STAT metadata table (nih-sra-datastore.sra_tax_analysis_tool.tax_analysis) and only get accessions that include Monkeypox virus (tax id = 10244). You could also cast a wider net and filter to Orthopox (10242) since many reads will not map unambiguously to Monkeypox and will be assigned to Orthopox. Second, we are going to JOIN this table with the SRA metadata table (nih-sra-datastore.sra.metadata) on the Accession number, this gives us more information about our record. Finally, we are going to only keep samples with at least 50 reads (total_count) assigned to the target tax id. This means that at least 50 reads need to be assigned to Monkeypox or daughter nodes in the phylogeny. Also we are going to look for records submitted anytime in the past 5 years (INTERVAL parameter).

In [ ]:
# Define the query
query = """
#standardSQL
WITH
  span AS (
  SELECT
    acc,
    ileft AS rileft,
    iright AS riright
  FROM
    nih-sra-datastore.sra_tax_analysis_tool.tax_analysis
  WHERE
    tax_id = 10244 )
SELECT
  acc,
  name,
  total_count,
  self_count,
  center_name,
  librarysource,
  organism,
  releasedate,
  geo_loc_name_sam,
  attributes
FROM
  nih-sra-datastore.sra_tax_analysis_tool.tax_analysis
JOIN
  nih-sra-datastore.sra.metadata
USING
  (acc)
JOIN
  span
USING
  (acc)
WHERE
  (ileft>=rileft
    AND iright<=riright)
  AND (total_count>99
    OR organism='wastewater metagenome')
  AND CAST(releasedate AS date) > DATE_SUB(CURRENT_DATE(), INTERVAL 120 month)
ORDER BY
  releasedate DESC,
  acc,
  total_count DESC
"""

Execute the query

In [ ]:
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

df = query_job.to_dataframe()

See how many unique accessions are in the df

In [ ]:
len(df['acc'].unique())

Now view the data frame. You will see that we first have the STAT metadata and then the SRA metadata such as submitter-assigned organism name, library source, release date etc. In the STAT metadata there are a few things to note. First, each tax id (name) with reads assigned from that accession is listed with the accession. So if the accession SRR12345 had reads assigned to both Monkeypox virus and variola virus, then you would see two records of that accession with both virus' listed under name. Next we have the numbers of reads (in NCBI parlance 'spots') assigned to that taxon id. We see two counts: total_count and self_count. Total count refers to the number of reads assigned to that node in the phylogeny (taxon) and all daughter nodes (decendents in the phylogeny). Self count refers to the number of reads assigned to that particular taxon. For example, if we had filtered more broadly to Orthopox (genus) instead of Monkeypox, we would have seen that Orthopox would have a total count of say 100 reads, which includes all reads assigned to any taxon within Orthopox with a self count of say 50. Then Monkeypox could have a total count of 50 and self count of 50 (assuming it is the terminal taxon in the tree with no daughter taxa). These 50 reads would be included in the total count of Orthopox, but not the self count of Orthopox. You can see all the STAT metadata fields [here](https://www.ncbi.nlm.nih.gov/sra/docs/sra-cloud-based-taxonomy-analysis-table/).

In [ ]:
# look at the df
df

Now we have a pandas data frame and you can filter and manipulate as desired.

In [ ]:
print(df['organism'].unique())

In [ ]:
# for example we can look at just wastewater samples 
df_filt = df[df['organism'] == 'wastewater metagenome']
df_filt

You can also write to an outfile.

In [ ]:
# Specify the columns to be written to the text file
columns_to_write = ["acc","name","total_count","self_count","center_name","librarysource","organism","releasedate","geo_loc_name_sam"]

# Write the specified columns to a text file
output_file = 'stat_results_mpox.txt'
df[columns_to_write].to_csv(output_file, sep='\t', index=False)

If you want to experiment a bit, rerun the query with a different tax id, modify the total_count, and modify the time Interval and see how your results change. Or, we can run a few more example queries from the [NCBI STAT page](https://www.ncbi.nlm.nih.gov/sra/docs/sra-cloud-based-taxonomy-analysis-table/). 

## Conclusion
Here you ran SQL queries using BigQuery to view STAT tables hosted in the cloud. 

## Clean Up
Feel free to delete the Vertex AI instance where you ran this notebook